In [ ]:
!pip install torch-geometric==2.5.0
!pip install e3nn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)


2.3.1+cu121
12.1


In [ ]:
import os
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import coalesce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cage Dataset

In [ ]:
class KbDatasetcage(InMemoryDataset):
    def __init__(self, root, split='044', transform=None, pre_transform=None, pre_filter=None):

        super().__init__(root, transform, pre_transform, pre_filter)
        self.root = root
        if split == '044':
          self.data, self.slices = torch.load(self.processed_paths[0])
        elif split=='050':
          self.data, self.slices = torch.load(self.processed_paths[1])
        elif split=='056':
          self.data, self.slices = torch.load(self.processed_paths[2])
        elif split=='064':
          self.data, self.slices = torch.load(self.processed_paths[3])

    @property
    def raw_file_names(self):
        return glob.glob(os.path.join(self.raw_dir,'Cage/data*'))

    @property
    def processed_file_names(self):
        # return ['train_data_inh_044_all.pt',]
        return ['train_data_cage044.pt','train_data_cage050.pt', 'train_data_cage056.pt','train_data_cage064.pt']

    def process(self):

        splits = ['44', '50', '56', '64']
        i= 0
        for split in splits:

          print(i)
          print('Cage'+'_'+split)
          path = os.path.join(self.raw_dir,f'Cage/T0.{split}')
          datalist = self.read_data(path,split)

          torch.save(self.collate(datalist), self.processed_paths[i])
          i+=1

    def read_data(self,p_path, split,max_files = 400):
        datalist =[]


        for i in range(1,101):
            print(i)
            path = os.path.join(p_path,f'train/N4096T0.{split}_{i}.npz')
            print(path)
            npz_data = np.load(path)
            data = process_graph(npz_data)
            datalist.append(data)

        for i in range(401,501):
            print(i)
            path  =os.path.join(p_path,f'test/N4096T0.{split}_{i}.npz')
            print(path)
            npz_data = np.load(path)
            data = process_graph(npz_data)
            datalist.append(data)

        return datalist

def process_graph(npz_data):
    num_nodes = 4096        # number of particles: fixed
    phi = 1.2                      # number density of the system
    box = np.full(3, (float(num_nodes) / phi )**(1.0/3.0) ) # length of simulation box
    type = npz_data['types']
    type_mask = type == 0
    target = npz_data['md_prop']
    target2 = npz_data['bb_prop']
    ### node feature
    x = torch.from_numpy(type).to(torch.long)
    positions = npz_data['initial_positions_cage']

    cross_positions = positions[None, :, :] - positions[:, None, :]
    box_ = box[None, None, :]
    cross_positions += (cross_positions < -box_ / 2.).astype(np.float32) * box_
    cross_positions -= (cross_positions > box_ / 2.).astype(np.float32) * box_
    distances = np.linalg.norm(cross_positions, axis=-1)
    indices = np.where( (distances < 2.0) & (distances > 1e-6))

    edge_index_inh = torch.from_numpy(np.array(indices)).to(torch.long)
    pos = torch.from_numpy(npz_data['initial_positions_inherent']).to(torch.float)
    pe = compute_pe(pos,box_, x)
###########################################################################
    positions = npz_data['initial_positions']

    cross_positions2 = positions[None, :, :] - positions[:, None, :]
    box_ = box[None, None, :]
    cross_positions2 += (cross_positions2 < -box_ / 2.).astype(np.float32) * box_
    cross_positions2 -= (cross_positions2 > box_ / 2.).astype(np.float32) * box_
    distances = np.linalg.norm(cross_positions2, axis=-1)
    indices = np.where( (distances < 2.0) & (distances > 1e-6))
    # cross_pos_in= torch.from_numpy(cross_positions).to(torch.float)
    # edge_weight = torch.from_numpy(distances[indices]).to(torch.float)
    edge_index_in = torch.from_numpy(np.array(indices)).to(torch.long)
    print(edge_index_in.shape)
    edge_index = torch.cat([edge_index_in,edge_index_inh], dim=-1)

    edge_index =  coalesce(edge_index)
    j,i = edge_index
    indices = (j.numpy(), i.numpy())

    edge_attr_inh = torch.from_numpy(cross_positions[indices]).to(torch.float)
    print(edge_index.shape)

    edge_attr_in = torch.from_numpy(cross_positions2[indices]).to(torch.float)

    box = torch.from_numpy(box)
    initial_pos = torch.from_numpy(npz_data['initial_positions']).to(torch.float)
    cage_pos = torch.from_numpy(npz_data['initial_positions_cage']).to(torch.float)
    dist_cage = cage_pos - initial_pos
    dist_cage += (dist_cage <  -box / 2.)* box
    dist_cage -= (dist_cage > box / 2.)* box


    data = Data(x = x,edge_index=edge_index,edge_attr_in = edge_attr_in,edge_attr_inh = edge_attr_inh,
          pos =dist_cage,pe = pe,y2=torch.from_numpy(target2.T).to(torch.float),
                            y=torch.from_numpy(target.T).to(torch.float))
    return data
def compute_pe(positions, box_, type):
  # Transposes edge features, so that the axes are in the first dimension.
  epsion_11 = 1.0
  epsion_12 = 1.5
  epsion_22 = 0.5
  sigma_11 = 1.0
  sigma_12 = 0.8
  sigma_22 = 0.88

  r_11 = 2.5* sigma_11
  r_12 = 2.5* sigma_12
  r_22 = 2.5 * sigma_22
  box_ = torch.from_numpy(box_)

  cross_positions = positions[None, :, :] - positions[:, None, :]
  # box_ = box[None, None, :]
  cross_positions += (cross_positions < -box_ / 2.) * box_
  cross_positions -= (cross_positions > box_ / 2.) * box_
  distances = torch.norm(cross_positions, dim=-1)

  # distances = torch.from_numpy(distances)
  types = type[:,None]+type
  mask_11 = (distances <r_11) & (distances > 1e-6) & (types ==2)
  masked_dis_11 = 2* epsion_11 * (((sigma_11/(distances+1e-6))* mask_11)**12 - ((sigma_11/(distances+1e-6))*mask_11)**6)

  mask_12 = (distances <r_12) & (distances > 1e-6) & (types ==3)
  masked_dis_12 = 2* epsion_12 * (((sigma_12/(distances+1e-6))* mask_12)**12 - ((sigma_12/(distances+1e-6))*mask_12)**6)

  mask_22 = (distances <r_22) & (distances > 1e-6) & (types ==4)
  masked_dis_22 = 2* epsion_22 * (((sigma_22/(distances+1e-6))* mask_22)**12 - ((sigma_22/(distances+1e-6))*mask_22)**6)

  PE = masked_dis_11.sum(dim = -1) + masked_dis_12.sum(dim = -1) + masked_dis_22.sum(dim = -1)

  return PE

#Dataloader

In [ ]:
root = '/content/drive/MyDrive'
train_dataset = KbDatasetcage(root=root,split='044')

train_loader = DataLoader(train_dataset[:100], batch_size=1, shuffle=True)
test_loader = DataLoader(train_dataset[100:], batch_size=1, shuffle=False)


# Model

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/EIGNN_github')
import os
os.chdir('/content/drive/MyDrive/EIGNN_github')

from model.eignn import EIGNN

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from e3nn.o3 import Irreps
edge_attr_irreps = Irreps('1x0e+1x1e+1x2e')


model = EIGNN(irreps_node = edge_attr_irreps,out_channels = 20,  irreps_edge=edge_attr_irreps, hidden_channels = 32,
        middle_channels =32,num_convs = 7).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001,betas = [0.95, 0.999], weight_decay = 1e-6)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,70,90], gamma=0.5)


/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


ElementwiseTensorProduct(3x0e x 1x0e+1x1e+1x2e -> 1x0e+1x1e+1x2e | 3 paths | 0 weights)
1x0e+1x0e+1x0e+1x1e+1x1e+1x1e+1x1e+1x1e+1x1e+1x2e+1x2e+1x2e+1x2e+1x2e+1x2e
3


/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: Us

ElementwiseTensorProduct(3x0e x 1x0e+1x1e+1x2e -> 1x0e+1x1e+1x2e | 3 paths | 0 weights)
1x0e+1x0e+1x0e+1x1e+1x1e+1x1e+1x1e+1x1e+1x1e+1x2e+1x2e+1x2e+1x2e+1x2e+1x2e
3


/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: Us

...

/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: Us

ElementwiseTensorProduct(3x0e x 1x0e+1x1e+1x2e -> 1x0e+1x1e+1x2e | 3 paths | 0 weights)
1x0e+1x0e+1x0e+1x1e+1x1e+1x1e+1x1e+1x1e+1x1e+1x2e+1x2e+1x2e+1x2e+1x2e+1x2e
3


/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: Us

ElementwiseTensorProduct(3x0e x 1x0e+1x1e+1x2e -> 1x0e+1x1e+1x2e | 3 paths | 0 weights)
1x0e+1x0e+1x0e+1x1e+1x1e+1x1e+1x1e+1x1e+1x1e+1x2e+1x2e+1x2e+1x2e+1x2e+1x2e
3


/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


# Training EIGNN-Cage

In [ ]:


def train(train_loader,st, end):
    model.train()
    total_loss = 0
    lens = end-st
    for i, data in enumerate(train_loader):

        data = data.to(device)
        y=data.y
        y = y[:,st:end]


        mask=data.x==1
        mask2=data.x==2
        y1 = (y[mask]-y[mask].mean(dim =0))/y[mask].std(dim =0)
        y2 = (y[mask2]-y[mask2].mean(dim =0))/y[mask2].std(dim =0)


        optimizer.zero_grad()
        s, edge= model(data)
        loss = F.mse_loss(s[mask,:lens],y1)+F.mse_loss(s[mask2,lens:],y2)

        loss.backward()
        total_loss += loss.item() * data.num_nodes
        optimizer.step()
        if (i + 1) % 20 == 0:
          print(f'[{i+1}/{len(train_loader)}] Loss: {loss.item()*data.num_nodes:.4f}')

    return total_loss / len(train_loader)
from scipy.stats import pearsonr


def test_mean(loader,st,end):
    model.eval()
    correct = []

    for data in loader:
        data = data.to(device)

        with torch.no_grad():
            pred,edge = model(data)

        mask=data.x==1


        mask=mask.cpu().numpy()

        pe= np.zeros(end-st)
        pe_o = np.zeros(end-st)
        for time_index in range(st,end):
          index = time_index-st
          y=data.y
          y =y[:,time_index]
          y = y.cpu().numpy()

          pe[index] = pearsonr(pred[mask,index].reshape(-1).cpu().numpy(),y[mask])[0]

        correct.append(pe)


    return correct


In [ ]:

st = 0
end =10
pcc_all = np.zeros(end-st)
for epoch in range(0, 100):
  loss = train(train_loader,st = 0, end = 10)
  if (epoch+1)%1==0:

    checkpoint = {
        "net": model.state_dict(),
        'optimizer':optimizer.state_dict(),
        "epoch": epoch,

    }
    torch.save(checkpoint, '/content/drive/MyDrive/t044_EIGNN_cage.pt')
  if (epoch+1)%1==0:
    pccs = test_mean(test_loader, st, end)
    pcc_mean = np.array(pccs).mean(0)

    # all_pred, all_true = test_all(test_loader_batch, st, end)
    # for t in range(end-st):
    #   pcc_all[t] = pearsonr(all_pred[t,:], all_true[t,:])[0]
    print(f'Epoch {epoch:03d}')
    print(pcc_mean)
    print(pcc_all)


  print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')
  scheduler.step()

[20/100] Loss: 6076.5923
[40/100] Loss: 5345.1772
[60/100] Loss: 4573.1729
[80/100] Loss: 4407.4849
[100/100] Loss: 4720.0229
Epoch 000
[0.34246936 0.81831238 0.87060599 0.77332053 0.74156758 0.74053923
 0.73942154 0.67233047 0.50433414 0.24339305]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Epoch 000, Loss: 5409.1545
[20/100] Loss: 4593.8506
[40/100] Loss: 4245.4580
[60/100] Loss: 4123.7520
[80/100] Loss: 4107.9229
[100/100] Loss: 4321.7480
Epoch 001
[0.37432755 0.84616767 0.88964339 0.80124895 0.77356895 0.76662311
 0.75773054 0.69462264 0.53256875 0.26898245]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Epoch 001, Loss: 4244.4877
[20/100] Loss: 3789.9470
[40/100] Loss: 3764.9102
[60/100] Loss: 4212.0879
[80/100] Loss: 4236.7988
[100/100] Loss: 3850.9238
Epoch 002
[0.37680731 0.85065506 0.89714689 0.80781224 0.7801651  0.77710338
 0.76450477 0.70921702 0.55131799 0.28288444]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Epoch 002, Loss: 4093.8611
[20/100] Loss: 4188.3926
[40/100] Loss: 3646.7300
[60/100] Loss: 3571.9885
[

#Test EIGNN-Cage

## Loading the checkpoint

In [ ]:
path_checkpoint ='/content/drive/MyDrive/t044_EIGNN_cage.pt' #
checkpoint = torch.load(path_checkpoint)  #
model.load_state_dict(checkpoint['net'])  #

<All keys matched successfully>

## Measuring PCC(mean) with 1 graphs in each batch

In [ ]:
test_loader_one = DataLoader(train_dataset[100:], batch_size=1, shuffle=False)

In [ ]:

pccs = test_mean(test_loader_one, 0, 10)
pcc_mean = np.array(pccs).mean(0)
print(per)

[0.87364581 0.87064157 0.91661659 0.86220094 0.84399332 0.8393544
 0.82280353 0.75662464 0.58931166 0.30577556]


##Measuring PCC(all) with 10 graphs in each batch

In [ ]:
test_loader_batch = DataLoader(train_dataset[100:], batch_size=10, shuffle=False)

In [ ]:
NTest=100
N=4096
NA=3277
NT=10
def test_final(loader,st,end):
    model.eval()
    all_pred = np.zeros((end-st,NTest*NA), dtype=float)
    all_true = np.zeros((end-st,NTest*NA), dtype=float)


    i = 0
    for data in loader:
        data = data.to(device)
        batch = data.batch
        max = data.batch.max() +1

        with torch.no_grad():
            pred,edge = model(data)

        mask=data.x==1

        mask=mask.cpu().numpy()

        pred = pred[mask,:end-st].T
        y = data.y[mask,st:end].T

        all_pred[:,i*NA*max:(i+1)*NA*max] = pred.cpu().numpy()
        all_true[:,i*NA*max:(i+1)*NA*max] =y.cpu().numpy()

        i+=1

    return all_pred, all_true

In [ ]:
st = 0
end = 10
pcc_all = np.zeros(end-st)
all_pred, all_true = test_all(test_loader_batch, st, end)
for t in range(end-st):
  pcc_all[t] = pearsonr(all_pred[t,:], all_true[t,:])[0]
print(pcc_all)

[0.87378081 0.87046921 0.91590845 0.86153974 0.84486262 0.8401456
 0.8226806  0.74941987 0.55762555 0.29931875]
